In [1]:
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd
import numpy as np
import os
import sys
import copy
import json
import sys


In [2]:
#Outputs all of the Stages, Counts, & Countries (Ex. Engage->Qualify)
if __name__ == "__main__":

    output_df = pd.DataFrame()

   

    directory = os.getcwd() + "/data"
    files = []
    for i in sorted(os.listdir(directory)):
        if os.path.isfile(os.path.join(directory,i)) and ('Product_Pipe' in i) and ('~' not in i):
            files.append(i)
    print(files)

 
#     if put in two excel sheets in commandline
#     if(len(sys.argv)>1):
#         #files = [sys.argv[1], sys.argv[2], sys.argv[3]]
#         files = sys.argv[1:]
#     else:
#         files = sorted(os.listdir(directory))
    

#     print(files)

    

    #first sheet we do value counts
    #prev = first sheet dataframe
    filename = files[0]
    

    f = os.path.join(directory, filename)
    print(f)
    df_prev = pd.read_excel(f, sheet_name='Sheet1')
    df_prev = pd.DataFrame(df_prev, columns=['Opp No', 'Sales Stage Name', 'Country', 'Opportunity Source', 'LIO Channel', 'Sector'])
    df_prev = df_prev.drop_duplicates(subset=["Opp No"], keep="last")
   

    filename = filename.split(".")[0][-5:]
 

    for i in range(1, len(files)):
        filename = files[i]
        f = os.path.join(directory, filename)
        df_current = pd.read_excel(f, sheet_name='Sheet1')
        df_current = pd.DataFrame(df_current, columns=['Opp No', 'Sales Stage Name', 'Country', 'Opportunity Source', 'LIO Channel', 'Sector'])
        print(df_current.head(3))
        df_current = df_current.drop_duplicates(subset=["Opp No"], keep="last")
        temp = df_current["Sales Stage Name"].value_counts()
        
        df_current = df_current.rename(columns= {'Opp No': "Opp No Cur", 'Sales Stage Name': "Sales Stage Name Cur", 'Country': "Country Cur", 'Opportunity Source': "Opp Source Cur", 'LIO Channel': "LIO Channel Cur", 'Sector': "Sector Cur"})
        df_compare = pd.merge(df_prev, df_current, how = "outer", left_on=df_prev["Opp No"], right_on = df_current["Opp No Cur"])

        df_compare = df_compare.fillna(0)
        df_compare.to_excel("test.xlsx")
        df_temp = pd.read_excel('engage.xlsx')
        temp2 = pd.DataFrame(df_temp)
        temp2.to_excel('temp2.xlsx')
        L = []
      
    
        newOpp_list = pd.DataFrame(columns=['Sales Stage Name'])

        #outputName = "compare" + str(i) + ".xlsx"
        #df_compare.to_excel(outputName)


        for index, row in df_compare.iterrows():

            Opp_No_prev = row["Opp No"]
            Opp_No_cur = row["Opp No Cur"]
            Country = row["Country Cur"]
            Country_prev = row["Country"]
            Opp_Source_prev = row["Opportunity Source"]
            Sector_prev = row["Sector"]
            LIO_Channel_prev = row["LIO Channel"]
            Opp_Source = row["Opp Source Cur"]
            Sector = row["Sector Cur"]
            LIO_Channel = row["LIO Channel Cur"]
            

            Sales_Stage_Name_prev = row["Sales Stage Name"]
            Sales_Stage_Name_cur = row["Sales Stage Name Cur"]

           
            if(Sales_Stage_Name_prev == "Qualify" and Sales_Stage_Name_cur==0):
                #loss
                key1 = "Loss"
                key2 = "Qualify->Loss"
                file = filename.split(".")[0][-5:]
                dat= [{'Date': file, 'Changes': key2, 'Changes_Cond': key1, 'Location': Country_prev, 'Source': Opp_Source_prev, 'Owner': LIO_Channel_prev, 'Sector': Sector_prev, 'Count': '1'}]
                dd = pd.DataFrame(dat)
                L.append(dd)
             
        
        
                
                
            elif(Sales_Stage_Name_prev == "Qualify" and Sales_Stage_Name_cur!=0):
                key1 = Sales_Stage_Name_cur
                key2 = "Qualify->" + Sales_Stage_Name_cur
                file = filename.split(".")[0][-5:]
                dat= [{'Date': file, 'Changes': key2, 'Changes_Cond': key1, 'Location': Country, 'Source': Opp_Source, 'Owner': LIO_Channel, 'Sector': Sector, 'Count': '1'}]
                dd = pd.DataFrame(dat)
                L.append(dd)
                    
                    
                
       #outside of loop
        df = pd.concat(L)
        df_temp = pd.read_excel('qualify.xlsx', index_col=[0])
        df_temp = pd.DataFrame(df_temp)
        frames = [df_temp, df]
        df_output = pd.concat(frames, ignore_index=True)
        df_output.to_excel('qualify.xlsx')
        df_current = df_current.rename(columns= {'Opp No Cur': "Opp No", 'Sales Stage Name Cur': "Sales Stage Name", 'Country Cur': "Country", 'Opp Source Cur': "Opportunity Source", 'LIO Channel Cur': "LIO Channel", 'Sector Cur': "Sector"})
        df_prev = df_current
        
    
 
        
               
   

['Product_Pipe_09-27.xlsx', 'Product_Pipe_10-04.xlsx', 'Product_Pipe_10-11.xlsx', 'Product_Pipe_10-18.xlsx', 'Product_Pipe_10-25.xlsx', 'Product_Pipe_11-01.xlsx']
/Users/Jessica.Journeycake@ibm.com/Desktop/R2M_UI/data/Product_Pipe_09-27.xlsx
   Opp No Sales Stage Name        Country Opportunity Source  LIO Channel  \
0       1           Engage  United States        Field Sales  Field Sales   
1       2           Engage  United States        Field Sales  Field Sales   
2       3           Engage  United States        Field Sales  Field Sales   

               Sector  
0  Energy & Utilities  
1  Energy & Utilities  
2  Energy & Utilities  
   Opp No Sales Stage Name        Country Opportunity Source  LIO Channel  \
0       1           Engage  United States        Field Sales  Field Sales   
1       2          Qualify  United States        Field Sales  Field Sales   
2       3          Qualify  United States        Field Sales  Field Sales   

               Sector  
0  Energy & Utilitie

In [3]:
#Engage Stats
df2 = pd.read_excel('qualify.xlsx', index_col=False)
df2 = pd.DataFrame(df2)
print(df2.head(5))
df2 = df2.drop(columns=['Unnamed: 0', 'Source', 'Owner', 'Sector', 'Date'])

df2 = df2["Changes"].value_counts()
df2 = pd.DataFrame(df2)
df2 = df2.rename(columns={"Changes": "Count"})
df2.to_excel('qualify_stats.xlsx')
df2 = pd.read_excel('qualify_stats.xlsx', index_col=False)
df2 = pd.DataFrame(df2)
df2 = df2.rename(columns={'Unnamed: 0': "Changes"})
df2.to_excel('qualify_stats.xlsx')

   Unnamed: 0   Date            Source             Owner Sector  \
0           0  10-04       Field Sales       Field Sales      0   
1           1  10-04       Field Sales       Field Sales      0   
2           2  10-04       Field Sales       Field Sales      0   
3           3  10-04  Business Partner  Business Partner      0   
4           4  10-04  Business Partner  Business Partner      0   

        Location Changes_Cond           Changes  Count  
0  United States      Qualify  Qualify->Qualify      1  
1  United States      Qualify  Qualify->Qualify      1  
2  United States       Design   Qualify->Design      1  
3  United States      Qualify  Qualify->Qualify      1  
4  United States      Qualify  Qualify->Qualify      1  


In [4]:
#Engage by Location
df2 = pd.read_excel('qualify.xlsx', index_col=False)
df2 = pd.DataFrame(df2)
df2 = df2.drop(columns=['Unnamed: 0', 'Source', 'Owner', 'Sector', 'Date'])
# df2.Location = df2.Location.str.strip()
df2 = df2.groupby(['Location', 'Changes'], as_index=False)["Count"].sum()
df2.to_excel('engage_location.xlsx')

In [5]:
#Engage by Owner
df2 = pd.read_excel('qualify.xlsx', index_col=False)
df2 = pd.DataFrame(df2)
df2 = df2.drop(columns=['Unnamed: 0', 'Location', 'Source', 'Sector', 'Date'])
df2.Owner = df2.Owner.str.strip()
df2 = df2.groupby(['Owner', 'Changes'], as_index=False)["Count"].sum()
df2.to_excel('qualify_owner.xlsx')

In [6]:
#Engage by Sector
df2 = pd.read_excel('qualify.xlsx', index_col=False)
df2 = pd.DataFrame(df2)
df2 = df2.drop(columns=['Unnamed: 0', 'Location', 'Source', 'Owner', 'Date'])
df2.Sector = df2.Sector.str.strip()
df2 = df2.groupby(['Sector', 'Changes'], as_index=False)["Count"].sum()
df2.to_excel('qualify_sector.xlsx')